# Introduction
Investigating Conversational AI for VSim

### TO DO'S

- if sound is unclear give proper responds
- if question not understood give proper responds
- Question: What if more matches happen? different matches?
- Can we save data for training more complex models? GPDR
- how should sound files be merged
- Medical terms
- Scalability? https://azure.microsoft.com/da-dk/pricing/details/cognitive-services/speech-services/

### Imports
Import libraries and write settings here.

In [1]:
import azure.cognitiveservices.speech as speechsdk
import json
import os
import spacy
import random
from spacy.tokens import Token
from playsound import playsound


#Load the Language Model
print('Initializing language model...')
nlp = spacy.load('en_core_web_lg') #large language model
#nlp = spacy.load('en_core_web_sm') #small language model
print('done..')
#Set credential for Microsft Cog Services
region = "eastus"
subkey = "3f7c2e668a5a4f3ca4923ff200e2db8b"

Initializing language model...
done..


### get paths and general questions

In [3]:
questions = []
pathToSoundFiles = []
answerDict = {}

#Getting dirs(what is named as the questions) and reponds.wav paths
for root, dirs, files in os.walk("./static/SoundFilesCAIPrototype/", topdown=False):
    for name in dirs:
        questions.append(name)
        answerDict[name] = {} 
   
    for name in files:
        p = os.path.join(root, name)      
        pathToSoundFiles.append(p)
      
#Creating a dict where each question contains the path for the proper responses  
for k in answerDict.keys():
    k_list = [key for key in pathToSoundFiles if k in key]
    
    answerDict[k] = k_list
    
print('The Questions Are', questions)

The Questions Are ['AreYouAllergicToAnything', 'CanYouDescribeThePain', 'DoYouHaveAnyPain', 'HowAreYouFeeling', 'PleaseStateYourNameAndDateOfBirth', 'WhatDoYouNeed']


In [ ]:
mapEventToQuesion = { 'AnyAllergies' : 'AreYouAllergicToAnything',
                     'DescirbePain' : 'CanYouDescribeThePain',
                      'HowFeel':'HowAreYouFeeling',
                      'NameAndBirth' : 'PleaseStateYourNameAndDateOfBirth',
                      'WhatYouNeed' : 'WhatDoYouNeed',
                      'AnyPain' : 'DoYouHaveAnyPain'

### Defining functions

In [3]:
def write_transcription(string):
    import uuid
    root_path = './transcriptions/'
    filename = root_path + str(uuid.uuid1())+'.txt'
    text_file = open(filename, "w")
    n = text_file.write(string)
    text_file.close()
    return

In [4]:
def write_json(string, label):
    import uuid
    import json
    root_path = './transcriptions/'
    filename = root_path + str(uuid.uuid1())+'.json'
    data = {'transcription': string ,'label': label}
    with open(filename, 'w') as fp:
        json.dump(data, fp)

    return

In [5]:
def see_text_files():
    text_paths = []
    strings = []
    for root, dirs, files in os.walk("./transcriptions/", topdown=False):
        for name in files:
            p = os.path.join(root, name)      
            text_paths.append(p)
    for t in text_paths:
        f = open(t, "r")
        strings.append(f.read())
    return strings


In [6]:
def what_match(d, matcher):
    matches=matcher(d)
    if matches:
        return doc.vocab.strings[matches[0][0]]
    else:
        return 'The question is not recognized. Please ask again.'

In [7]:
def display_doc(doc):
    from spacy import displacy
    displacy.render(doc, style="dep", jupyter=True)
    return

In [8]:
def merge_sound_files(path_file1, path_file2, dest_path):
    from pydub import AudioSegment

    sound1 = AudioSegment.from_mp3(path_file1)
    sound2 = AudioSegment.from_mp3(path_file2)

    combined_sounds = sound1 + sound2
    combined_sounds.export(dest_path, format="mp3")

# Chat Bot or Question Inten Module

### Choose Question

In [23]:
# Test Question 1
doc = nlp('Can you please state your name and the day you were born')
doc.to_json()

{'text': 'Can you please state your name and the day you were born',
 'ents': [{'start': 35, 'end': 42, 'label': 'DATE'}],
 'sents': [{'start': 0, 'end': 56}],
 'tokens': [{'id': 0,
   'start': 0,
   'end': 3,
   'pos': 'VERB',
   'tag': 'MD',
   'dep': 'aux',
   'head': 3},
  {'id': 1,
   'start': 4,
   'end': 7,
   'pos': 'PRON',
   'tag': 'PRP',
   'dep': 'nsubj',
   'head': 3},
  {'id': 2,
   'start': 8,
   'end': 14,
   'pos': 'INTJ',
   'tag': 'UH',
   'dep': 'intj',
   'head': 3},
  {'id': 3,
   'start': 15,
   'end': 20,
   'pos': 'VERB',
   'tag': 'VB',
   'dep': 'ROOT',
   'head': 3},
  {'id': 4,
   'start': 21,
   'end': 25,
   'pos': 'DET',
   'tag': 'PRP$',
   'dep': 'poss',
   'head': 5},
  {'id': 5,
   'start': 26,
   'end': 30,
   'pos': 'NOUN',
   'tag': 'NN',
   'dep': 'dobj',
   'head': 3},
  {'id': 6,
   'start': 31,
   'end': 34,
   'pos': 'CCONJ',
   'tag': 'CC',
   'dep': 'cc',
   'head': 3},
  {'id': 7,
   'start': 35,
   'end': 38,
   'pos': 'DET',
   'tag': 'D

### Generate Patterns

#### Allergies

In [10]:
anyAllergies = [
    {'POS': {"IN": ["AUX","PART"]}},
    {'POS': 'PRON'},
    {'POS': {"IN": ["AUX","PART","DET"]}, 'OP': '*'},
    {'lemma': {"IN": ['allergic','allergy','hypersensitivity']}}  
    ]

#### Can you describe the pain

In [11]:
DescirbePain = [
    {'POS': {"IN": ['PRON']}},
    {'POS': {"IN": ["AUX","PART","DET"]}, 'OP': '*'},
    {'lemma': {'IN' : ['describe','characterize','explain','identify']}},
    {'POS': {"IN": ["AUX","PART","DET"]}, 'OP': '*'},
    {'lemma': {'IN' : ['pain']}}
    ]

#### Do you have any pain

In [12]:
AnyPain = [
    {'POS': {"IN": ['PRON']}},
    {'lemma': {'IN' : ['have','are']}, 'OP': '?'},
    {'POS': {"IN": ["AUX","PART","DET","ADP"]}, 'OP': '*'},
    {'lemma': {'IN' : ['pain']}}
    ]

#### How are you feeling

In [13]:
HowFeel = [
    {'lemma': 'how'},
    {'POS': 'AUX'},
    {'POS': { 'IN' : ['PRON','DET']}},
    {'lemma': {'IN' : ['feel','do','go']}, 'OP' : '?'}
    ]

#### Please state your name and date of birth

In [48]:
NameAndBirth = [
    {'lower': 'your'},
    {'lemma': 'name'},
    {'POS': {"IN": ["AUX","PART","DET","ADP","NOUN",'CCONJ','PRON']}, 'OP': '*'},
    {'ORTH': {'IN' : ['birth','birthday','born']}}
    ]

#### What do you need

In [49]:
WhatYouNeed = [
    {'lower': 'what'},
    {'lemma': {'IN' : ['do','is']}},
    {'POS': {"IN": ["AUX","PART","DET","ADP","NOUN",'CCONJ','PRON']}, 'OP': '*'},
    {'lemma': {'IN' : ['need','want','require']}}
    ]

### Find Matches

In [50]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

matcher.add('AnyAllergies', None, anyAllergies )
matcher.add('DescirbePain', None, DescirbePain )
matcher.add('AnyPain', None, AnyPain )
matcher.add('HowFeel', None, HowFeel )
matcher.add('NameAndBirth', None, NameAndBirth )
matcher.add('WhatYouNeed', None, WhatYouNeed )


matches = matcher(doc)

for match_id, start, end in matches:
    matched_span = doc[start:end]
    print('matched span:', matched_span.text)
what_match(doc, matcher)

matched span: your name and the day you were born


'NameAndBirth'

In [24]:
display_doc(doc)

# STT Module


In [2]:
speech_key, service_region = subkey, region
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)

#speech_config.request_word_level_timestamps()

In [19]:
result = speech_recognizer.recognize_once()
print(result.text)

Hello, my name is Daniel Beck Hansen. I'm born in Denmark.


## Conversational Model

In [51]:
def ConversationalModel(nlp=nlp, matcher=matcher, speech_recognizer=speech_recognizer):
    print('Start Speaking..')
    result = speech_recognizer.recognize_once()
    print('Recognized:',result.text)
    print('Matched event:', what_match(nlp(result.text),matcher))
    
    if what_match(nlp(result.text),matcher) == 'AnyAllergies':
        playsound(random.choice(answerDict[questions[0]]))
    elif what_match(nlp(result.text),matcher) == 'DescirbePain':
        playsound(random.choice(answerDict[questions[1]]))
    elif what_match(nlp(result.text),matcher) == 'AnyPain':
        playsound(random.choice(answerDict[questions[2]]))
    elif what_match(nlp(result.text),matcher) == 'HowFeel':
        playsound(random.choice(answerDict[questions[3]]))
    elif what_match(nlp(result.text),matcher) == 'NameAndBirth':
        playsound(random.choice(answerDict[questions[4]]))
    elif what_match(nlp(result.text),matcher) == 'WhatYouNeed':
        playsound(random.choice(answerDict[questions[5]]))
    else:
        print('Question not recognized')
    
    write_transcription(result.text)
    write_json(result.text, what_match(nlp(result.text),matcher))

In [55]:
ConversationalModel()

Start Speaking..
Recognized: Do you have any allergies?
Matched event: AnyAllergies


# TTS Module

In [6]:
audio_config = speechsdk.AudioOutputConfig(filename="./test1.wav")


In [7]:
synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
synthesizer.speak_text_async("A simple test to write to a file.")

In [30]:
# output sound directly
# audio_config = speechsdk.AudioOutputConfig(use_default_speaker=True)
# synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)
# synthesizer.speak_text_async("A simple test to write to a file.")

In [32]:
synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=None)
result = synthesizer.speak_text_async("Getting the response as an in-memory stream.").get()
stream = speechsdk.AudioDataStream(result)

# Results
Show graphs and stats here

In [12]:
speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config)


In [13]:
result = speech_recognizer.recognize_once()


# Logging Data

In [149]:
import numpy
ms = nlp.vocab.vectors.most_similar(numpy.asarray([nlp.vocab.vectors[nlp.vocab.strings['need']]]), n=200)
[nlp.vocab.strings[w] for w in ms[0][0]]

['need',
 'Need',
 'NEED',
 'WANT',
 'wAnt',
 'want',
 'Want',
 'NEEDED',
 'Needed',
 'needed',
 'CAn',
 'cAN',
 'CaN',
 'can',
 'cAn',
 'Can',
 'CAN',
 'Needs',
 'NEEDS',
 'needs',
 'NEEDs',
 'SHOULD',
 'SHould',
 'should',
 'Should',
 'Get',
 'gEt',
 'GET',
 'gET',
 'get',
 'GEt',
 'GeT',
 'ABLE',
 'able',
 'Able',
 'YoU',
 'You',
 'YOU',
 'yoU',
 'you',
 'yOU',
 'yOu',
 'YOu',
 'MAKE',
 'make',
 'Make',
 'mAke',
 'MAke',
 'SURE',
 'Sure',
 'sure',
 'SUre',
 'do',
 'Do',
 'DO',
 'dO',
 'GIVE',
 'give',
 'Give',
 'GIve',
 'KNOW',
 'knOw',
 'Know',
 'know',
 'KNow',
 'HAve',
 'havE',
 'haVe',
 'have',
 'hAVE',
 'Have',
 'HAVE',
 'hAve',
 'HaVe',
 'Needing',
 'needing',
 'NEEDING',
 'COULD',
 'COuld',
 'Could',
 'could',
 "'ll",
 "'LL",
 "'Ll",
 'Must',
 'must',
 'MUST',
 'WOuld',
 'wOuld',
 'WOULD',
 'would',
 'Would',
 "N'T",
 "n't",
 "n'T",
 "N't",
 'Help',
 'help',
 'HELP',
 'might',
 'Might',
 'MIGHT',
 'MIght',
 'KEEP',
 'Keep',
 'keep',
 'ENOUGH',
 'Enough',
 'enough',
 'understa

In [4]:
import spacy
from sense2vec import Sense2VecComponent

nlp = spacy.load("en_core_web_sm")
s2v = Sense2VecComponent(nlp.vocab)
nlp.add_pipe(s2v)

doc = nlp("A sentence about natural language processing.")
assert doc[3:6].text == "natural language processing"
freq = doc[3:6]._.s2v_freq
vector = doc[3:6]._.s2v_vec
most_similar = doc[3:6]._.s2v_most_similar(3)
# [(('machine learning', 'NOUN'), 0.8986967),
#  (('computer vision', 'NOUN'), 0.8636297),
#  (('deep learning', 'NOUN'), 0.8573361)]

ValueError: Can't find key natural_language_processing|NOUN in table

In [13]:
nlp.vocab[u'dog'].cluster

37

In [16]:
vector1 = nlp(u"dog").vector
result = nlp.vocab.vectors.most_similar(vector1.reshape(1,vector1.shape[0]))

In [ ]:
import numpy as np
import pandas as pd
import spacy
from tqdm import tqdm
import hdbscan

nlp = spacy.load('en_core_web_lg')

"""define all vectors and strings"""
strings = []
vectors = []
for key, vector in tqdm(nlp.vocab.vectors.items(), total = len(nlp.vocab.vectors.keys())):
    try:
        strings.append(nlp.vocab.strings[key])
        vectors.append(vector)
    except:
        pass

vectors = np.vstack(vectors)

"""Generate clusters, you will need to save the clusterer to use as 
clustering can take a while"""
clusterer = hdbscan.HDBSCAN(min_cluster_size = 1000)
clusterer.fit(vectors)
labels = clusterer.labels_


#main function
def closest(word, count = 10):
    
    word = nlp(word)
    main = word.vector
    
    cluster = clusterer.fit(main).labels_
    tmp_vectors = vectors[np.where(labels == cluster)[0]]    
    tmp_strings = np.array(strings)[np.where(labels == cluster)[0]]
    
    diff = tmp_vectors - main
    diff = diff**2
    diff = np.sqrt(diff.sum(axis = 1), dtype = np.float64)
   
    df = pd.DataFrame(tmp_strings, columns = ['keyword'])
    df['diff'] = diff
    df = df.sort_values('diff', ascending = True).head(count)
    df['keyword'] = df['keyword'].str.lower()
    df = df.drop_duplicates(subset = 'keyword', keep = 'first')
    return df

closest("Dogs", count = 10)

100%|██████████████████████████████████████████████████████████████████████| 684830/684830 [00:03<00:00, 218971.97it/s]


# Conclusions and Next Steps
Summarize findings here

In [16]:
dir(result)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_cancellation_details',
 '_duration',
 '_error_json',
 '_json',
 '_no_match_details',
 '_offset',
 '_properties',
 '_reason',
 '_result_id',
 '_text',
 'cancellation_details',
 'duration',
 'error_json',
 'json',
 'no_match_details',
 'offset',
 'properties',
 'reason',
 'result_id',
 'text']

In [24]:
result.text

'Hello hello hello hello, my name is Daniel how are you doing?'

In [21]:
data = json.loads(result.json)

In [22]:
data

{'DisplayText': 'Hello hello hello hello, my name is Daniel how are you doing?',
 'Duration': 45500000,
 'Id': '784bf512ecb64666b145fd13abad4fb2',
 'NBest': [{'Confidence': 0.8401363,
   'Display': 'Hello hello hello hello, my name is Daniel how are you doing?',
   'ITN': 'hello hello hello hello my name is daniel how are you doing',
   'Lexical': 'hello hello hello hello my name is daniel how are you doing',
   'MaskedITN': 'hello hello hello hello my name is daniel how are you doing',
   'Words': [{'Duration': 7700000, 'Offset': 26700000, 'Word': 'hello'},
    {'Duration': 4600000, 'Offset': 35200000, 'Word': 'hello'},
    {'Duration': 7400000, 'Offset': 40100000, 'Word': 'hello'},
    {'Duration': 4800000, 'Offset': 47800000, 'Word': 'hello'},
    {'Duration': 1300000, 'Offset': 52700000, 'Word': 'my'},
    {'Duration': 2000000, 'Offset': 54100000, 'Word': 'name'},
    {'Duration': 2000000, 'Offset': 56200000, 'Word': 'is'},
    {'Duration': 5500000, 'Offset': 58300000, 'Word': 'dan